In [11]:
import os
import csv
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

def leer_puntos_desde_csv(archivo_csv):
    """
    Lee un archivo CSV con puntos de latitud y longitud y prepara la lista de puntos para el polígono.

    Parámetros:
    - archivo_csv: Ruta del archivo CSV que contiene los puntos.

    Retorna:
    - Lista de tuplas (longitud, latitud) que representan los puntos del polígono.
    """
    df = pd.read_csv(archivo_csv)
    puntos = list(zip(df['Longitud'], df['Latitud']))
    return puntos

def leer_archivo(path):
    """Intentar leer el archivo con diferentes codificaciones."""
    try:
        return pd.read_csv(path, encoding='utf-8', low_memory=False)
    except UnicodeDecodeError:
        try:
            return pd.read_csv(path, encoding='latin1', low_memory=False)
        except UnicodeDecodeError:
            print(f"Error leyendo el archivo {path} con 'utf-8' y 'latin1'.")
            return None

def filtrar_por_poligono(df, puntos, crs="EPSG:4326"):
    """Filtra el DataFrame por las coordenadas dentro de un polígono definido por una lista de tuplas (lat, lon)."""
    
    # Verificar si las columnas de longitud y latitud existen
    if 'Longitud' not in df.columns or 'Latitud' not in df.columns:
        raise ValueError("Las columnas 'Longitud' y 'Latitud' no se encuentran en el DataFrame.")
    
    # Crear un polígono usando los puntos proporcionados
    poligono = Polygon(puntos)  # Asume que 'puntos' es una lista de tuplas (lon, lat)
    # Crear GeoDataFrame desde el DataFrame de entrada
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitud, df.Latitud))
    # Establecer el CRS para el GeoDataFrame
    gdf.set_crs(crs, inplace=True)
    
    # Crear una GeoSerie para el polígono con el mismo CRS
    poligono_gdf = gpd.GeoSeries([poligono], crs=crs)
    # Filtrar los datos que están dentro del polígono
    datos_filtrados = gdf[gdf.geometry.within(poligono_gdf.unary_union)]
    
    # Opcionalmente, convertir de nuevo a DataFrame normal eliminando la columna 'geometry'
    datos_filtrados = datos_filtrados.drop(columns='geometry')
    return datos_filtrados

def procesar_archivo_filtrado(indice, path, carpeta_salida, poligono):
    try:
        # Corrige las rutas usando raw strings o dobles barras invertidas
        base_path = r"C:\Users\ricar\OneDrive\Documentos\Proyectos\Oceana\proyecto"
        file_path = os.path.join(base_path, path)

        if path.endswith('.csv'):
            df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv
            if df is None:
                return
        elif path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Archivo no compatible: {path}")
            return
        
        # Asumiendo que la función 'filtrar_por_poligono' está definida
        df_filtrado = filtrar_por_poligono(df, poligono)
        
        if df_filtrado.empty:
            print(f"No se encontraron datos dentro del polígono para el archivo {path}.")
            return
        
        print(f"Se encontraron {len(df_filtrado)} registros dentro del polígono para el archivo {path}.")
        
        nombre_archivo = os.path.basename(path)
        nombre_archivo_filtrado = f"filtrado_{nombre_archivo}"
        output_path = os.path.join(carpeta_salida, nombre_archivo_filtrado)
        if path.endswith('.csv'):
            df_filtrado.to_csv(output_path, index=False, encoding='utf-8')
        elif path.endswith('.xlsx'):
            df_filtrado.to_excel(output_path, index=False)
        
        print(f"Archivo filtrado y guardado: {output_path}")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo: {str(e)}")
    
    except Exception as e:
        print(f"Error procesando el archivo {path}: {e}")

def filtrar_archivos_por_poligono(archivo_lista_paths, carpeta_salida, puntos):
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)
    
    # Crear el polígono irregular
    poligono_irregular = Polygon(puntos)
    
    with open(archivo_lista_paths, mode='r') as file:
        reader = csv.reader(file)
        next(reader)
        paths = [(indice, path) for indice, path in reader]
    
    for indice, path in paths:
        procesar_archivo_filtrado(indice, path, carpeta_salida, poligono_irregular)




In [12]:
archivo_lista_paths = r"C:\Users\ricar\OneDrive\Documentos\Proyectos\Oceana\proyecto\outputs\lista_de_archivos_vms_homogenizados.csv"  # Reemplaza con la ruta de tu archivo de entrada
carpeta_salida = 'archivos_filtrados'  # Reemplaza con la ruta de tu carpeta de salida

archivo_puntos_csv = 'vms_vertices.csv'
puntos = leer_puntos_desde_csv(archivo_puntos_csv)

filtrar_archivos_por_poligono(archivo_lista_paths, carpeta_salida, puntos)

C:\Users\ricar\AppData\Local\Temp\ipykernel_66208\1512343715.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv


Se encontraron 34708 registros dentro del polígono para el archivo archivos_homogenizados\01 - 01 -15 ENE  2023.csv.
Archivo filtrado y guardado: archivos_filtrados\filtrado_01 - 01 -15 ENE  2023.csv
Se encontraron 34708 registros dentro del polígono para el archivo archivos_homogenizados\01 - 01 -15 ENE  2023.xlsx.
Archivo filtrado y guardado: archivos_filtrados\filtrado_01 - 01 -15 ENE  2023.xlsx


C:\Users\ricar\AppData\Local\Temp\ipykernel_66208\1512343715.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv


Se encontraron 38247 registros dentro del polígono para el archivo archivos_homogenizados\01 - 16 -29 ENE  2023.csv.
Archivo filtrado y guardado: archivos_filtrados\filtrado_01 - 16 -29 ENE  2023.csv
Se encontraron 38247 registros dentro del polígono para el archivo archivos_homogenizados\01 - 16 -29 ENE  2023.xlsx.
Archivo filtrado y guardado: archivos_filtrados\filtrado_01 - 16 -29 ENE  2023.xlsx


C:\Users\ricar\AppData\Local\Temp\ipykernel_66208\1512343715.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv


...

Se encontraron 30195 registros dentro del polígono para el archivo archivos_homogenizados\21-31-OCT-2019.csv.
Archivo filtrado y guardado: archivos_filtrados\filtrado_21-31-OCT-2019.csv


C:\Users\ricar\AppData\Local\Temp\ipykernel_66208\1512343715.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv


Se encontraron 5868 registros dentro del polígono para el archivo archivos_homogenizados\26-30-SEP-2018.csv.
Archivo filtrado y guardado: archivos_filtrados\filtrado_26-30-SEP-2018.csv


C:\Users\ricar\AppData\Local\Temp\ipykernel_66208\1512343715.py:62: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Asumiendo que existe una función pd.read_csv


Se encontraron 25611 registros dentro del polígono para el archivo archivos_homogenizados\26-31-OCT-2018.csv.
Archivo filtrado y guardado: archivos_filtrados\filtrado_26-31-OCT-2018.csv
Ocurrió un error al procesar el archivo: Las columnas 'Longitud' y 'Latitud' no se encuentran en el DataFrame.
